In [ ]:
pip install etna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.1/387.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.0/354.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
pip install clickhouse_driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.4/740.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import re
from datetime import datetime, date, time
import math
from etna.pipeline import Pipeline
from etna.datasets import TSDataset
from etna.pipeline.pipeline import Pipeline
from etna.models import (
    NaiveModel,
    SeasonalMovingAverageModel,
    CatBoostMultiSegmentModel,
)
from etna.transforms import LagTransform, LinearTrendTransform, LogTransform, MedianOutliersTransform, TimeSeriesImputerTransform, DateFlagsTransform
from etna.transforms import HolidayTransform, StandardScalerTransform, FourierTransform, MeanTransform, TrendTransform, YeoJohnsonTransform
from etna.metrics import MAE, MSE, SMAPE, MAPE
from etna.analysis.outliers import get_anomalies_median, get_anomalies_density
from ruptures.detection import Binseg
from etna.models import CatBoostMultiSegmentModel
from etna.models import SARIMAXModel
from etna.metrics import SMAPE, MAE, MAPE, MSE
from etna.analysis import plot_backtest, plot_trend, plot_forecast, plot_anomalies, plot_change_points_interactive, plot_imputation, plot_feature_relevance
from etna.analysis import plot_periodogram, stl_plot, seasonal_plot, find_change_points, plot_time_series_with_change_points
from etna.analysis import ModelRelevanceTable
from etna.ensembles import VotingEnsemble, StackingEnsemble
from catboost import CatBoostRegressor
from clickhouse_driver import Client
from functools import partial
import random
import wandb
from typing import Optional
import typer

In [ ]:
class DataHandler:
    def __init__(self, date, url, source):
        self.date = date
        self.url = url
        self.source = source

    def get_data(self):
        client = Client('big-dw1.hse.ru',
                user='USER',
                password='PASSWORD',
                secure=True,
                verify=False,
                database='web_analytics',
                compression=True)
        list_to_str=','.join(self.source)
        res=re.sub(r',', '|', list_to_str)
        sql1 = '''SELECT source, url, ts, COUNT(DISTINCT(user_id)) AS users FROM web_analytics.yandex_raw_data
        WHERE ts >= '%s' AND source REGEXP ('%s') AND match(url, '%s') GROUP BY ts, url, source ORDER BY ts'''% (self.date, res, self.url)
        result, columns = client.execute(sql1, with_column_types=True)
        data= pd.DataFrame(result, columns = [tuple[0] for tuple in columns])
        grouped_df=data.groupby(['url', 'ts']).agg({'users': 'sum'}).reset_index()
        r=grouped_df['url'].tolist()
        for i in r:
            if 'https' in i:
                r[r.index(i)] = 0
        grouped_df['url']=r
        filtered=grouped_df.loc[grouped_df['url'] != 0]
        return filtered

In [ ]:
class TSDatasetGenerator:
    def __init__(self, filtered):
        self.filtered = filtered

    def generate_ts_dataset(self):
        self.filtered["timestamp"] = self.filtered["ts"]
        self.filtered["target"] = self.filtered["users"]
        self.filtered.drop(columns=["ts", "users"], inplace=True)
        self.filtered = self.filtered.rename(columns={'url': 'segment'})
        return TSDataset.to_dataset(self.filtered), self.filtered

class TSDatasetGeneratorNew:
    def __init__(self, filtered, segment):
        self.filtered = filtered
        self.segment = segment

    def generate_ts_dataset(self):
        self.filtered["timestamp"] = self.filtered["ts"]
        self.filtered["target"] = self.filtered["users"]
        self.filtered.drop(columns=["ts", "users"], inplace=True)
        self.filtered = self.filtered.rename(columns={'url': 'segment'})
        return TSDataset.to_dataset(self.filtered), self.filtered

class Ts(DataHandler, TSDatasetGenerator, TSDatasetGeneratorNew):
    def ts(self):
        pass

class TsTrend:
    def __init__(self, ts, poly_degree_e):
        self.ts = ts
        self.poly_degree_e = poly_degree_e

    def get_trend(self):
        trends = [
            LinearTrendTransform(in_column="target", poly_degree=1),
            LinearTrendTransform(in_column="target", poly_degree=self.poly_degree_e)
        ]
        return plot_trend(self.ts, trend_transform=trends)

class TsSeasonality:
    def __init__(self, ts, cycle):
        self.ts = ts
        self.cycle = cycle

    def get_seasonality(self):
        return seasonal_plot(ts=self.ts, cycle=self.cycle)

class Plot(TsTrend, TsSeasonality):
      def plot(self):
          pass